In [ ]:
# IMport the library and the functions
import gym
from DQN import DQN, plot_history
from keras import Sequential
from keras.layers import Dense, Input
from Falsification import MoonlightFalsification

In [ ]:
# Define the environment
env = gym.make('LunarLander-v2')
# Define the number of actions
num_actions = env.action_space.n
# Define the state size
state_size = env.observation_space.shape[0]

# Define the q network
q_network = Sequential([
    Input(state_size),
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=num_actions, activation='linear')
])

# Define the target q network
target_q = Sequential([
    Input(state_size),
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=num_actions, activation='linear')
])

# Define the class of DQN 
dqn = DQN(learning_rate=0.001, size_batch=64, epsilon_dec=0.6)

# Compute the DQN and create the model
cDR_list, cDR_mean_list = dqn.DQN(env=env, q_network=q_network, target_q=target_q, max_episodes=5000)

# Plot the result of the CDR 
plot_history(cDR_list, cDR_mean_list)

q_network.save('NoName_Model.h5')

In [ ]:
# Define the Moonlight Class
falsification = MoonlightFalsification(model_path="Lander_trained_model.h5")

# Initialize the number of falsification and the list to save the results
n_falsification_runs = 5
falsifications_lists = [[] for _ in range(n_falsification_runs)]
best_individual_lists = [[] for _ in range(n_falsification_runs)]

# Execute the simulations 
for yy in range(n_falsification_runs):
    # Compute the best individuals for all simulations and save the individuals that falsify the formulas
    best_individual_lists[yy], falsifications_lists[yy] = falsification.falsification( 
        exp_name=f"NoName_Run-{yy}of{n_falsification_runs}", n_parents=10, n_offspring_per_parent=2,
        simulation_steps=500, max_falsification_evaluations=2000, mutation_rate=0.05)
    # Save the data in a csv structure
    best_individual_lists[yy].to_csv(f'NoName_best-individuals_run-{yy+1}of{n_falsification_runs}.csv', index=False)
    falsifications_lists[yy].to_csv(f'NoName_falsification-list_run-{yy+1}of{n_falsification_runs}.csv', index=False)